# Seattle Freeway Traffic Speed Field

The traffic speed field data is a matrix of size $75\times 4032$.

In [ ]:
import numpy as np
np.random.seed(1000)
plt.rcParams['font.size'] = 12

dense_tensor = np.load('../datasets/Seattle-data-set/tensor.npz')['arr_0']
dim = dense_tensor.shape
dense_mat = dense_tensor.reshape([dim[0], dim[1] * dim[2]])

import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize = (12, 2.5))
sns.heatmap(dense_mat[165 : 240, 0 * 288 : 14 * 288], cmap='jet_r',
            cbar_kws={'label': 'Traffic speed'}, vmin = 0, vmax = 75)
plt.xticks(np.arange(0, 288 * 14 + 1, 288), np.arange(0, 288 * 14 + 1, 288), rotation = 0)
plt.yticks(np.arange(0.5, 75.5, 10), np.arange(166, 241, 10), rotation = 0)
plt.xlabel('Time')
plt.ylabel('Loop detector')
plt.show()
fig.savefig("Seattle_speed_field.png", bbox_inches = "tight")

### LCR without Spatial Kernel

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - N * T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR_2d(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [ ]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Seattle-data-set/tensor.npz')['arr_0']
dim = dense_tensor.shape
missing_rate = 0.3

sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1])[:, :, np.newaxis] + 0.5 - missing_rate)
dense_mat = dense_tensor.reshape([dim[0], dim[1] * dim[2]])
sparse_mat = sparse_tensor.reshape([dim[0], dim[1] * dim[2]])
del dense_tensor, sparse_tensor

dense_mat = dense_mat[165 : 240, 0 * 288 : 14 * 288]
sparse_mat = sparse_mat[165 : 240, 0 * 288 : 14 * 288]

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 1 * lmbda
tau = 1
maxiter = 100
mat_hat = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

### LCR with Both Spatial and Temporal Kernels

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - N * T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR_2d(y_true, y, lmbda, gamma, tau_s, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = laplacian(N, tau_s)
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [ ]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Seattle-data-set/tensor.npz')['arr_0']
dim = dense_tensor.shape
missing_rate = 0.3

sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1])[:, :, np.newaxis] + 0.5 - missing_rate)
dense_mat = dense_tensor.reshape([dim[0], dim[1] * dim[2]])
sparse_mat = sparse_tensor.reshape([dim[0], dim[1] * dim[2]])
del dense_tensor, sparse_tensor

dense_mat = dense_mat[165 : 240, 0 * 288 : 14 * 288]
sparse_mat = sparse_mat[165 : 240, 0 * 288 : 14 * 288]

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 1 * lmbda
tau_s = 1
tau = 1
maxiter = 100
mat_hat = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau_s, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize = (12, 2.5))
sns.heatmap(mat_hat, cmap='jet_r', cbar_kws={'label': 'Traffic speed'}, vmin = 0, vmax = 75)
plt.xticks(np.arange(0, 288 * 14 + 1, 288), np.arange(0, 288 * 14 + 1, 288), rotation = 0)
plt.yticks(np.arange(0.5, 75.5, 10), np.arange(166, 241, 10), rotation = 0)
plt.xlabel('Time')
plt.ylabel('Loop detector')
plt.show()

# NGSIM Speed Field Reconstruction

## LCR

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def laplacian_qv(T):
    ell = np.zeros(T)
    ell[0] = 1
    ell[-1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - N * T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR_2d(y_true, y, lmbda, gamma, tau1, tau2, maxiter = 50, qv = False):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = ~np.isnan(y)
    y_train = y[pos_train]
    pos_test = np.where((y_true > 0) & (np.isnan(y)))
    y_test = y_true[pos_test]
    y[np.isnan(y)] = 0
    z = y.copy()
    w = y.copy()
    if qv == False:
        ell_s = laplacian(N, tau1)
        ell_t = laplacian(T, tau2)
    else:
        ell_s = laplacian_qv(N)
        ell_t = laplacian_qv(T)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

import numpy as np
np.random.seed(1)

import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

dense_mat = np.load('../datasets/NGSIM-data-set/NGSIM_full.npy')
sparse_mat = np.load('../datasets/NGSIM-data-set/NGSIM_80missing.npy')

def plot_speed_field(data, filename):
    fig = plt.figure(figsize = (2.5 * 2.5, 2.5))
    plt.matshow(data, cmap='jet_r', origin='lower', 
                vmin = 0, vmax = 25, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xticks([0, 100, 200, 300, 400, 500], [0, 500, 1000, 1500, 2000, 2500])
    plt.yticks([0, 100, 200], [0, 300, 600])
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (mph)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

plot_speed_field(dense_mat, 'speed_field_fully_data.png')
plot_speed_field(sparse_mat, 'speed_field_80_missing_data.png')

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-4 * N * T
gamma = 5 * lmbda
tau1 = 2
tau2 = 2
maxiter = 100
mat_hat = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, 
                 tau1, tau2, maxiter, qv = False)
end = time.time()
print('Running time: %d seconds.'%(end - start))
plot_speed_field(mat_hat, 'speed_field_LCR_rec.png')

## LCR Without Spatiotemporal Laplacian Kernels

In [ ]:
import numpy as np
np.random.seed(1)

import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

dense_mat = np.load('../datasets/NGSIM-data-set/NGSIM_full.npy')
sparse_mat = np.load('../datasets/NGSIM-data-set/NGSIM_80missing.npy')

def plot_speed_field(data, filename):
    fig = plt.figure(figsize = (2.5 * 2.5, 2.5))
    plt.matshow(data, cmap='jet_r', origin='lower', 
                vmin = 0, vmax = 25, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xticks([0, 100, 200, 300, 400, 500], [0, 500, 1000, 1500, 2000, 2500])
    plt.yticks([0, 100, 200], [0, 300, 600])
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (mph)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

plot_speed_field(dense_mat, 'speed_field_fully_data.png')
plot_speed_field(sparse_mat, 'speed_field_80_missing_data.png')

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-4 * N * T
gamma = 0 * lmbda
tau1 = 2
tau2 = 2
maxiter = 100
mat_hat = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, 
                 tau1, tau2, maxiter, qv = False)
end = time.time()
print('Running time: %d seconds.'%(end - start))
plot_speed_field(mat_hat, 'speed_field_CircNNM_rec.png')

## Quadratic Variation Completion

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian_qv(T):
    ell = np.zeros(T)
    ell[0] = 1
    ell[-1] = -1
    return ell

def update_x(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    return np.fft.ifft2(temp).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def QVC(y_true, y, lmbda, gamma, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = ~np.isnan(y)
    y_train = y[pos_train]
    pos_test = np.where((y_true > 0) & (np.isnan(y)))
    y_test = y_true[pos_test]
    y[np.isnan(y)] = 0
    z = y.copy()
    w = y.copy()
    kernel = np.outer(laplacian_qv(N), laplacian_qv(T))
    denominator = lmbda + gamma * np.fft.fft2(kernel) * np.conjugate(np.fft.fft2(kernel))
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = update_x(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

import numpy as np
np.random.seed(1)

import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

dense_mat = np.load('../datasets/NGSIM-data-set/NGSIM_full.npy')
sparse_mat = np.load('../datasets/NGSIM-data-set/NGSIM_80missing.npy')

def plot_speed_field(data, filename):
    fig = plt.figure(figsize = (2.5 * 2.5, 2.5))
    plt.matshow(data, cmap='jet_r', origin='lower', 
                vmin = 0, vmax = 25, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xticks([0, 100, 200, 300, 400, 500], [0, 500, 1000, 1500, 2000, 2500])
    plt.yticks([0, 100, 200], [0, 300, 600])
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (mph)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

plot_speed_field(dense_mat, 'speed_field_fully_data.png')
plot_speed_field(sparse_mat, 'speed_field_80_missing_data.png')

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-4 * N * T
gamma = 5 * lmbda
maxiter = 100
mat_hat = QVC(dense_mat, sparse_mat, lmbda, gamma, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))
plot_speed_field(mat_hat, 'speed_field_QVC_rec.png')

## Laplacian Kernelized Completion

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def update_x(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    return np.fft.ifft2(temp).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LKC(y_true, y, lmbda, gamma, tau1, tau2, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = ~np.isnan(y)
    y_train = y[pos_train]
    pos_test = np.where((y_true > 0) & (np.isnan(y)))
    y_test = y_true[pos_test]
    y[np.isnan(y)] = 0
    z = y.copy()
    w = y.copy()
    kernel = np.outer(laplacian(N, tau1), laplacian(T, tau2))
    denominator = lmbda + gamma * np.fft.fft2(kernel) * np.conjugate(np.fft.fft2(kernel))
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = update_x(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

import numpy as np
np.random.seed(1)

import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

dense_mat = np.load('../datasets/NGSIM-data-set/NGSIM_full.npy')
sparse_mat = np.load('../datasets/NGSIM-data-set/NGSIM_80missing.npy')

def plot_speed_field(data, filename):
    fig = plt.figure(figsize = (2.5 * 2.5, 2.5))
    plt.matshow(data, cmap='jet_r', origin='lower', 
                vmin = 0, vmax = 25, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xticks([0, 100, 200, 300, 400, 500], [0, 500, 1000, 1500, 2000, 2500])
    plt.yticks([0, 100, 200], [0, 300, 600])
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (mph)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

plot_speed_field(dense_mat, 'speed_field_fully_data.png')
plot_speed_field(sparse_mat, 'speed_field_80_missing_data.png')

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-4 * N * T
gamma = 5 * lmbda
tau1 = 2
tau2 = 2
maxiter = 100
mat_hat = LKC(dense_mat, sparse_mat, lmbda, gamma, tau1, tau2, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))
plot_speed_field(mat_hat, 'speed_field_LKC_rec.png')

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>